In [1]:
import ipywidgets as ipw
import subprocess

In [2]:
from utils.control import *

In [3]:
msg,updates_needed,config = check_for_updates('s1267')
#tutto = check_for_updates()

In [4]:
 msg

"✅ The .ssh/config seems to be OK.<br>✅ Active AiiDA computers: ✅daint.alps_s1267<br>✅localhost<br>✅⬜ Not active AiiDA computers: ✅⬜202503061611_daint.alps_s1267<br>✅⬜202503070725_daint.alps_s1267<br>✅⬜Used_till_202502100738_daint.alps<br>✅⬜202503070715_daint.alps_s1267<br>✅⬜Used_till_202502100831_daint.alps<br>✅⬜daint.alps<br>✅⬜202503070700_daint.alps_s1267<br>✅⬜Used_till_202502120836_daint.alps<br>✅⬜202503061643_daint.alps_s1267<br>✅⬜Used_till_202502091226_daint.alps<br>✅⬜Used_till_202502120922_daint.alps<br>✅⬜202503070709_daint.alps_s1267<br>✅⬜Used_till_202502100724_daint.alps<br>✅⬜Used_till_202502120713_daint.alps<br>✅ Active AiiDA codes:<br> ✅ dos@daint.alps  PK: 15196<br>✅ pp@daint.alps  PK: 15194<br>✅ ph-7.4@localhost  PK: 10254<br>✅ projwfc-7.2@localhost  PK: 1819<br>✅ pw2wannier90-7.2@localhost  PK: 1829<br>✅ blabla@daint.alps  PK: 15197<br>✅ pw-7.4:v2@daint.alps_s1267  PK: 15304<br>✅ projwfc-7.4@localhost  PK: 10247<br>✅ dos-7.4:v2@daint.alps  PK: 15188<br>✅ pw2gw-7.2@localho

In [5]:
config

{'python_version': '3.9.13',
 'new_host_label': 'daint.alps',
 'remotehost': 'daint.alps.cscs.ch',
 'proxy': 'ela.cscs.ch',
 'grants': {'daint.alps': ['s1267', 's1276', 's1295'], 'tigu': ['free']},
 'computers': {'daint.alps': {'setup': {'description': 'daint.alps at CSCS',
    'hostname': 'daint.alps.cscs.ch',
    'label': 'daint.alps_cscsaccount',
    'mpiprocs_per_machine': 256,
    'mpirun_command': 'srun --cpu-bind=socket /users/cscsusername/bin/mps-wrapper.sh',
    'prepend_text': '#SBATCH --ntasks-per-core=1\n#SBATCH --hint=nomultithread\n#SBATCH --no-requeue\n#SBATCH --account=cscsaccount\n',
    'scheduler': 'core.slurm',
    'shebang': '#!/bin/bash -l',
    'transport': 'core.ssh',
    'use_double_quotes': False,
    'work_dir': '/capstor/scratch/cscs/{username}/aiida/',
    'default_memory_per_machine': 268435456},
   'config': {'allow_agent': True,
    'compress': True,
    'gss_auth': False,
    'gss_deleg_creds': False,
    'gss_host': 'daint.alps.cscs.ch',
    'gss_kex':

In [6]:
updates_needed

{'codes': {'cp2k-2024.3:v2@daint.alps_s1267': {'rename': False,
   'install': True}}}